# Setup

In [1]:
# Standard library imports
import warnings
import os

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasetsforecast.hierarchical import HierarchicalData
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import  BottomUp, TopDown, MiddleOut, MinTrace, ERM
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuration & Settings
warnings.simplefilter(action='ignore', category=FutureWarning)

/Users/kbanachewicz/Documents/time-series-with-Konrad/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This section of code imports necessary libraries for time series forecasting, specifically focusing on hierarchical forecasting. 

It begins by importing modules from Python’s standard library: `warnings` to manage warning messages and `os` which provides a way of using operating system dependent functionality.

Next, it imports several third-party packages. `numpy` is fundamental for numerical operations, while `pandas` is used for data manipulation and analysis with DataFrames. `matplotlib.pyplot` enables the creation of visualizations like plots and charts. 

The code then brings in specific tools from the `datasetsforecast`, `hierarchicalforecast`, `statsforecast`, and `sklearn` packages. From `datasetsforecast`, it imports `HierarchicalData` for handling hierarchical data structures.  From `hierarchicalforecast`, several modules are imported: `aggregate` for aggregating time series, `HierarchicalReconciliation` which is the core class for reconciliation methods, and various reconciliation strategies like `BottomUp`, `TopDown`, `MiddleOut`, `MinTrace`, and `ERM`. The `statsforecast` package provides tools for statistical forecasting; it imports `StatsForecast` as a central class and specific models such as `AutoARIMA` (automatic ARIMA model selection) and `Naive` (a simple baseline forecast). Finally, `hierarchicalforecast.evaluation` is used to assess the performance of hierarchical forecasts, and functions from `sklearn.metrics` (`mean_absolute_error`, `mean_squared_error`) are included for calculating error metrics.

The final line suppresses `FutureWarning` messages, which can clutter output without necessarily indicating critical issues. This ensures a cleaner presentation of results during execution.

In [2]:
# general settings
class CFG:
    data_folder = './data/'
    graph_folder = './graphs/'
    img_dim1 = 20
    img_dim2 = 10
    SEED = 42
    metric = 'rmse'


# display style 
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (CFG.img_dim1, CFG.img_dim2)

np.random.seed(CFG.SEED)

This code segment defines general settings and adjusts the display style for subsequent operations, likely related to data analysis and visualization. 

A class named `CFG` is created to encapsulate configuration parameters. It specifies the directories for storing data (`data_folder`) and generated graphs (`graph_folder`).  It also sets dimensions for images (`img_dim1`, `img_dim2`), a random seed value (`SEED`) for reproducibility, and defines the evaluation metric to be used as root mean squared error (`metric`).

Following this, the code modifies the default plotting style using `matplotlib.pyplot`. It applies the "seaborn-v0_8" style theme and sets the default figure size based on the dimensions defined in the `CFG` class. 

Finally, it initializes the NumPy random number generator with the seed value specified in `CFG.SEED`, ensuring that any randomized processes will produce consistent results across multiple runs of the code.

# Utils

In [3]:
def forecast_metrics(y_true, y_pred, metric='rmse', precision = 2):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    if metric == 'rmse':
        return np.round(np.sqrt(mean_squared_error(y_true, y_pred)), precision)
    elif metric == 'mse':
        return np.round(mean_squared_error(y_true, y_pred), precision)
    elif metric == 'mae':
        return np.round(mean_absolute_error(y_true, y_pred), precision)
    else:
        raise ValueError(f"Unsupported metric: {metric}")

This code defines a function called `forecast_metrics` that calculates and returns forecast evaluation metrics.

The function takes two arguments: `y_true`, representing the actual values, and `y_pred`, representing the predicted values. It also accepts optional parameters for the desired `metric` (defaulting to 'rmse') and the number of decimal places for rounding (`precision`, defaulting to 2).

Inside the function, both input arrays are converted to NumPy arrays using `np.array()`.  A series of conditional statements then checks the value of the `metric` parameter. If it's 'rmse', the root mean squared error is calculated using `mean_squared_error` from scikit-learn, and the square root is taken before rounding to the specified precision. Similarly, if `metric` is 'mse', the mean squared error is directly computed and rounded.  If `metric` is 'mae', the mean absolute error is calculated and rounded. 

If an unsupported metric name is provided, a `ValueError` exception is raised with a descriptive message indicating which metric was not recognized. The function ultimately returns the calculated and rounded metric value.

In [4]:
def merge_forecasts(y_test_df, y_pred_df):
    return pd.merge(y_test_df, y_pred_df, on=['ds', 'unique_id'])

This code defines a function named `merge_forecasts` that combines actual and predicted time series data into a single DataFrame.

The function accepts two arguments: `y_test_df`, which is a pandas DataFrame containing the true (observed) values, and `y_pred_df`, a DataFrame holding the forecasted values. 

It uses the `pd.merge` function from the pandas library to join these two DataFrames based on common columns. The `on=['ds', 'unique_id']` argument specifies that the merge should be performed using the ‘ds’ ( representing date/timestamp) and ‘unique_id’ columns, which are assumed to uniquely identify each time series within the data. 

The function returns a new DataFrame where rows from `y_test_df` and `y_pred_df` are combined based on matching values in these specified columns, allowing for direct comparison of actual versus predicted values.

In [5]:
def compute_overall_error(df, actual_col, pred_col, metric='rmse'):
    return forecast_metrics(df[actual_col], df[pred_col], metric=metric)

This code defines a function named `compute_overall_error` that calculates an overall error metric for a given DataFrame.

The function accepts three arguments: `df`, which is the Pandas DataFrame containing both actual and predicted values; `actual_col`, specifying the name of the column in the DataFrame holding the true values; and `pred_col`, indicating the column with the forecasted values. It also takes an optional argument, `metric`, defaulting to 'rmse', that determines which error metric will be computed.

The function’s core functionality is a single line: it calls the previously defined `forecast_metrics` function, passing in the actual and predicted value columns from the DataFrame (`df[actual_col]` and `df[pred_col]`, respectively) along with the specified `metric`. The result returned by `forecast_metrics` – the calculated error score – is then directly returned by `compute_overall_error`. Essentially, this function serves as a convenient wrapper to apply the `forecast_metrics` calculation to specific columns within a DataFrame.

In [6]:
def compute_error_per_level(df, tags, actual_col, pred_col, metric='rmse'):
    results = []
    for level, ids in tags.items():
        mask = df['unique_id'].isin(ids)
        err = forecast_metrics(df.loc[mask, actual_col], df.loc[mask, pred_col], metric=metric)
        results.append({
            'level': level,
            f'{metric}': err
        })
    return pd.DataFrame(results)

This code defines a function called `compute_error_per_level` that calculates error metrics for each level within a hierarchical dataset.

The function takes five arguments: `df`, the Pandas DataFrame containing the data; `tags`, a dictionary where keys represent levels in the hierarchy and values are lists of unique identifiers belonging to each level; `actual_col`, the name of the column with actual values; `pred_col`, the name of the column with predicted values; and an optional `metric` argument (defaulting to 'rmse') specifying the error metric.

The function initializes an empty list called `results`. It then iterates through each level and its corresponding IDs in the `tags` dictionary. Inside the loop, a boolean mask is created using `df['unique_id'].isin(ids)`, which selects rows from the DataFrame where the 'unique_id' column matches any of the IDs for the current level. 

The `forecast_metrics` function is then called with the filtered actual and predicted values (obtained using `.loc[mask, actual_col]` and `.loc[mask, pred_col]`) to calculate the error metric for that specific level. A dictionary containing the level name and the calculated error value is created and appended to the `results` list.

Finally, after processing all levels, the function converts the `results` list into a Pandas DataFrame and returns it. This resulting DataFrame will have columns 'level' and the specified metric (e.g., 'rmse'), providing an overview of forecast accuracy at each hierarchical level.

In [7]:
def compare_baseline_vs_reconciled(df, tags, actual_col, baseline_col, reconciled_col, metric='rmse'):
    results = []
    for level, ids in tags.items():
        mask = df['unique_id'].isin(ids)

        baseline_err = forecast_metrics(df.loc[mask, actual_col], df.loc[mask, baseline_col], metric=metric)
        reconciled_err = forecast_metrics(df.loc[mask, actual_col], df.loc[mask, reconciled_col], metric=metric)

        results.append({
            'level': level,
            f'baseline_{metric}': baseline_err,
            f'reconciled_{metric}': reconciled_err,
            f'delta_{metric}': reconciled_err - baseline_err,
            'improved': reconciled_err < baseline_err
        })
    return pd.DataFrame(results)

This code defines a function called `compare_baseline_vs_reconciled` that compares the performance of a baseline forecast against a reconciled forecast at each level of a hierarchical structure.

The function takes six arguments: `df`, the Pandas DataFrame containing the data; `tags`, a dictionary defining the hierarchy (levels and their IDs); `actual_col`, the column with actual values; `baseline_col`, the column with baseline forecasts; `reconciled_col`, the column with reconciled forecasts; and an optional `metric` argument (defaulting to 'rmse') for evaluating performance.

It initializes an empty list called `results`. The code then iterates through each level and its associated IDs in the `tags` dictionary. Inside the loop, a boolean mask is created using `df['unique_id'].isin(ids)` to select data belonging to the current level. 

The `forecast_metrics` function is called twice: once to calculate the error between actual values and the baseline forecasts (`baseline_err`), and again to calculate the error between actual values and the reconciled forecasts (`reconciled_err`). Both calculations use the specified `metric`.

A dictionary is created containing the level name, the baseline error, the reconciled error, the difference (delta) between the two errors, and a boolean value indicating whether the reconciled forecast improved upon the baseline (i.e., if the reconciled error is less than the baseline error). This dictionary is appended to the `results` list.

Finally, after processing all levels, the function converts the `results` list into a Pandas DataFrame and returns it. The resulting DataFrame provides a level-by-level comparison of the baseline and reconciled forecasts, including the performance improvement (or degradation) achieved by reconciliation.

# Groundwork

In [8]:
# M5 competition data - aggregated and prepared in https://github.com/tng-konrad/time-series-with-Konrad/blob/main/helper-m04.ipynb
df = pd.read_csv(CFG.data_folder + 'm04_sales_data_prepared.csv')
# add a (dummy) column for total level
df['top_level'] = 'Total'
# hierarchy structure: mid level: middle_level, bottom level: bottom_level
df.rename(columns={'state_id': 'middle_level', 'store_id': 'bottom_level'}, inplace=True)
# arrange columns
df = df[['ds', 'top_level',  'middle_level', 'bottom_level', 'y']]
# format date column
df['ds'] = pd.to_datetime(df['ds'])

This code segment loads, prepares, and formats data for hierarchical forecasting, specifically using a dataset from the M5 competition.

It begins by reading a CSV file named `m04_sales_data_prepared.csv` located in the directory specified by `CFG.data_folder` (defined earlier as `./data/`) into a Pandas DataFrame called `df`. The data preparation steps used to create this CSV are documented in a GitHub notebook referenced in the comment.

Next, a new column named 'top_level' is added to the DataFrame and populated with the string "Total" for all rows. This creates a dummy top-level aggregation point for the hierarchical structure. 

The code then renames two existing columns: 'state_id' is renamed to 'middle_level', and 'store_id' is renamed to 'bottom_level'. These represent the intermediate and lowest levels of the hierarchy, respectively.

Following this, the order of columns in the DataFrame is rearranged to be `['ds', 'top_level', 'middle_level', 'bottom_level', 'y']`, where 'ds' represents the date column and 'y' represents the target variable (sales data). 

Finally, the 'ds' column is converted from its original data type to a datetime object using `pd.to_datetime()`. This ensures that time series operations can be performed correctly on this column.

In [9]:
df.head(3)

,ds,top_level,middle_level,bottom_level,y
0,2014-01-01,Total,CA,CA_1,3
1,2014-01-01,Total,CA,CA_2,0
2,2014-01-01,Total,CA,CA_3,7


In [31]:
# prepare hierarchical data
hierarchy_levels = [
    ["top_level"],  ["top_level", "middle_level"],  ["top_level", "middle_level", "bottom_level"] ]

Y_hier_df, S_df, tags = aggregate(df=df, spec=hierarchy_levels)

# print("S_df.shape", S_df.shape)
# print("Y_hier_df.shape", Y_hier_df.shape)

# quick check
for k in tags.keys():
    print('---')
    print(f"tags['{k}'] :", tags[k])
#     print(f"tags['{k}'] shape:", tags[k].shape)



---
tags['top_level'] : ['Total']
---
tags['top_level/middle_level'] : ['Total/CA' 'Total/TX' 'Total/WI']
---
tags['top_level/middle_level/bottom_level'] : ['Total/CA/CA_1' 'Total/CA/CA_2' 'Total/CA/CA_3' 'Total/CA/CA_4'
 'Total/TX/TX_1' 'Total/TX/TX_2' 'Total/TX/TX_3' 'Total/WI/WI_1'
 'Total/WI/WI_2' 'Total/WI/WI_3']


This code prepares the data for hierarchical forecasting using the `aggregate` function from the `hierarchicalforecast` library.

First, a list called `hierarchy_levels` is defined. This list specifies the levels of the hierarchy to be created. Each element in the list represents a level and contains the names of the columns that define that level. For example, `["top_level"]` defines the top-most level using only the 'top_level' column, while `["top_level", "middle_level", "bottom_level"]` defines the bottom-most level using all three hierarchical columns.

The `aggregate` function is then called with the DataFrame `df` and the `hierarchy_levels` specification. This function restructures the data into a format suitable for hierarchical forecasting, returning three objects: `Y_hier_df`, `S_df`, and `tags`.  `Y_hier_df` contains the time series data aggregated at each level of the hierarchy. `S_df` likely holds static information about the hierarchy structure. `tags` is a dictionary that maps each hierarchical level to a list of unique identifiers (IDs) belonging to that level.

The code then prints the shapes of `S_df` and `Y_hier_df` to provide an initial check on the data dimensions after aggregation. 

Finally, it iterates through the keys (levels) in the `tags` dictionary and prints each level's name along with the corresponding list of IDs and its shape. This provides a quick verification that the hierarchy has been constructed correctly and that each level contains the expected number of unique identifiers.

In [11]:
horizon = 7 

Y_test_df = Y_hier_df.groupby("unique_id", as_index=False).tail(horizon)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

This code segment splits the hierarchical time series data into training and testing sets based on a specified forecast horizon.

First, the `horizon` variable is set to 7, indicating that the forecasts will be made for the next 7 time steps.

Then, `Y_test_df` is created by grouping the `Y_hier_df` DataFrame (which contains the aggregated time series data) by 'unique_id' and selecting the last `horizon` rows from each group using `.tail(horizon)`. This effectively creates a test set consisting of the most recent 7 observations for each unique time series in the hierarchy. The `as_index=False` argument prevents the 'unique_id' column from becoming the index of the resulting DataFrame.

Finally, `Y_train_df` is created by removing the rows present in `Y_test_df` from `Y_hier_df` using `.drop(Y_test_df.index)`. This leaves a training set containing all time series data except for the last 7 observations of each series, which are reserved for testing.TODO

In [12]:
# Compute base auto-ARIMA predictions
model = AutoARIMA(season_length=7)
fcst = StatsForecast(models=[model], freq="D", n_jobs=-1)
Y_hat_df = fcst.forecast(df=Y_train_df, h=4, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

This code computes baseline forecasts using an AutoARIMA model and the `statsforecast` library.

First, an `AutoARIMA` model is initialized with a `season_length` of 7, indicating that the time series data exhibits weekly seasonality.

Next, a `StatsForecast` object is created. This object manages the forecasting process using one or more models. In this case, it's configured to use only the `AutoARIMA` model defined earlier. The `freq="D"` argument specifies that the data has daily frequency, and `n_jobs=-1` tells `statsforecast` to use all available CPU cores for parallel processing.

The `fcst.forecast()` method is then called with `Y_train_df` (the training data) as input and a forecast horizon of `h=4`. This generates forecasts for the next 4 time steps for each series in the training set. The `fitted=True` argument instructs the model to also store the fitted values from the training period.  The resulting DataFrame containing the forecasted values is stored in `Y_hat_df`.

Finally, `fcst.forecast_fitted_values()` retrieves the fitted values (in-sample predictions) generated by the AutoARIMA model during the training process and stores them in `Y_fitted_df`. These fitted values represent the model's best estimate of the historical data based on the training set.TODO

In [13]:
xmat = merge_forecasts(Y_test_df, Y_hat_df)

# keep track of the real / prediction column name
actual_col = 'y'
baseline_col = xmat.columns[-1]

xmat.head(3)

,unique_id,ds,y,AutoARIMA
0,Total,2016-05-16,12,14.092568
1,Total,2016-05-17,11,14.102201
2,Total,2016-05-18,22,14.102201


This code merges the actual test data with the baseline forecasts generated by the AutoARIMA model and prepares for comparison.

The `merge_forecasts` function is called with `Y_test_df` (the actual values from the test set) and `Y_hat_df` (the forecasted values). This function combines the two DataFrames, aligning them based on the 'unique_id' and time index to create a single DataFrame (`xmat`) containing both the true values and the corresponding forecasts.

The code then defines variables `actual_col` and `baseline_col`.  `actual_col` is set to 'y', which represents the column name for the actual observed values in the original data. `baseline_col` is assigned the name of the last column in the merged DataFrame (`xmat`), which corresponds to the forecasted values generated by the AutoARIMA model.

Finally, `xmat.head(3)` displays the first three rows of the merged DataFrame `xmat`. This allows for a quick inspection of the data structure and confirms that the actual and predicted values have been correctly aligned.

In [14]:
overall_rmse = forecast_metrics(xmat['y'], xmat[baseline_col], metric = CFG.metric)
print(f'Overall {CFG.metric} : {overall_rmse}')


for k, ids in tags.items():
    mask = xmat['unique_id'].isin(ids)
    k_metrics = forecast_metrics(xmat.loc[mask, 'y'], xmat.loc[mask, baseline_col])
    print(f"{k} {CFG.metric}: {k_metrics}")

Overall rmse : 2.15
top_level rmse: 5.06
top_level/middle_level rmse: 2.45
top_level/middle_level/bottom_level rmse: 1.45


This code calculates and prints overall and per-level evaluation metrics for the AutoARIMA baseline forecasts.

First, it computes the overall Root Mean Squared Error (RMSE) – or whatever metric is specified in `CFG.metric` – between the actual values (`xmat['y']`) and the baseline forecasts (`xmat[baseline_col]`) using the `forecast_metrics` function. The result is stored in the `overall_rmse` variable, and then printed to the console with a descriptive message.

Next, it iterates through each level of the hierarchy defined by the `tags` dictionary. Inside the loop, a boolean mask (`mask`) is created to select rows from the `xmat` DataFrame that belong to the current hierarchical level (based on 'unique_id'). 

The `forecast_metrics` function is then called with the actual values and baseline forecasts for the selected level (using `.loc[mask, 'y']` and `.loc[mask, baseline_col]`). The resulting metric value (`k_metrics`) is printed to the console along with the level name and the metric type specified in `CFG.metric`. This provides a breakdown of forecast accuracy at each level of the hierarchy.

# Hierarchical methods

## BottomUp

In [15]:
reconcilers = [BottomUp()] 

hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)

# column name of the reconciled forecasts
reconciled_col = Y_rec_df.columns[-1]

Y_rec_df.head(3)

,unique_id,ds,AutoARIMA,AutoARIMA/BottomUp
0,Total,2016-05-16,14.092568,13.943087
1,Total,2016-05-17,14.102201,13.327107
2,Total,2016-05-18,14.102201,13.409471


This code performs hierarchical reconciliation to improve forecast accuracy by leveraging the hierarchical structure of the data.

First, a list called `reconcilers` is created containing a single reconciliation method: `BottomUp()`. This indicates that the bottom-up reconciliation approach will be used, where forecasts are aggregated from lower levels to higher levels in the hierarchy.

Next, a `HierarchicalReconciliation` object (`hrec`) is initialized with the specified list of reconcilers. 

The core reconciliation step is then performed using `hrec.reconcile()`. This function takes four arguments: `Y_hat_df` (the forecasts from the base model), `Y_df` (the fitted values from the base model, used for adjusting the reconciled forecasts), `S_df` (static information about the hierarchy structure), and `tags` (the dictionary defining the hierarchical levels). The function reconciles the base forecasts to ensure consistency across all levels of the hierarchy.  The resulting DataFrame containing the reconciled forecasts is stored in `Y_rec_df`.

The code then determines the name of the column in `Y_rec_df` that contains the reconciled forecasts and assigns it to the variable `reconciled_col`. This is done by accessing the last column of the DataFrame. 

Finally, `Y_rec_df.head(3)` displays the first three rows of the reconciled forecast DataFrame, allowing for a quick inspection of the results.

In [16]:
# Merge reconciled forecasts with test data
df_eval = merge_forecasts(Y_test_df, Y_rec_df)

This code merges the actual test data (`Y_test_df`) with the reconciled forecasts (`Y_rec_df`) to prepare for evaluating the performance of the reconciliation process.

The `merge_forecasts` function is called with `Y_test_df` and `Y_rec_df` as arguments. This function combines the two DataFrames, aligning them based on the 'unique_id' and time index to create a single DataFrame (`df_eval`) containing both the true values from the test set and the corresponding reconciled forecasts. The resulting DataFrame is ready for calculating evaluation metrics that compare the reconciled forecasts against the actual observations.

In [17]:
metric = 'rmse'

baseline_overall = compute_overall_error(df_eval, actual_col, baseline_col, metric)
reconciled_overall = compute_overall_error(df_eval, actual_col, reconciled_col, metric)

print(f"Baseline {metric.upper()}: {baseline_overall:.4f}")
print(f"Reconciled {metric.upper()}: {reconciled_overall:.4f}")
print(f"Delta {metric.upper()}: {reconciled_overall - baseline_overall:.4f}")

Baseline RMSE: 2.1500
Reconciled RMSE: 2.1800
Delta RMSE: 0.0300


This code calculates and prints the overall evaluation metrics for both the baseline forecasts and the reconciled forecasts.

First, the `metric` variable is set to 'rmse', specifying that Root Mean Squared Error will be used as the evaluation metric.

Then, `compute_overall_error` is called twice: once to calculate the overall RMSE for the baseline forecasts (`baseline_overall`), using the actual values (`actual_col`) and the baseline forecast column (`baseline_col`); and again to calculate the overall RMSE for the reconciled forecasts (`reconciled_overall`), using the actual values and the reconciled forecast column (`reconciled_col`).

Finally, the code prints the calculated metrics to the console. It displays the baseline RMSE, the reconciled RMSE, and the difference (delta) between the two, formatted to four decimal places using f-strings (`:.4f`). This allows for a direct comparison of the performance of the baseline forecasts versus the reconciled forecasts across the entire dataset.

In [18]:
per_level_baseline = compute_error_per_level(df_eval, tags, actual_col, baseline_col, metric)
per_level_reconciled = compute_error_per_level(df_eval, tags, actual_col, reconciled_col, metric)


comparison_df = compare_baseline_vs_reconciled(
    df=df_eval,
    tags=tags,
    actual_col=actual_col,
    baseline_col=baseline_col,
    reconciled_col=reconciled_col,
    metric=metric
)

comparison_df

,level,baseline_rmse,reconciled_rmse,delta_rmse,improved
0,top_level,5.06,5.14,0.08,False
1,top_level/middle_level,2.45,2.52,0.07,False
2,top_level/middle_level/bottom_level,1.45,1.45,0.00,False


This code calculates and presents a detailed comparison of the baseline and reconciled forecasts at each level of the hierarchy.

First, `compute_error_per_level` is called twice: once to calculate the RMSE (or specified metric) for the baseline forecasts at each hierarchical level (`per_level_baseline`), and again to do the same for the reconciled forecasts (`per_level_reconciled`). Both calls use the evaluation DataFrame `df_eval`, the hierarchy definition in `tags`, and the appropriate forecast columns.

Next, the `compare_baseline_vs_reconciled` function is called with all necessary arguments: the evaluation DataFrame `df_eval`, the hierarchy structure `tags`, the actual value column name `actual_col`, the baseline forecast column name `baseline_col`, the reconciled forecast column name `reconciled_col`, and the chosen metric `metric`. This function generates a DataFrame (`comparison_df`) that summarizes the performance comparison at each level, including the baseline error, reconciled error, the difference between them (delta), and an indicator of whether reconciliation improved accuracy.

Finally, the code displays the contents of the `comparison_df` DataFrame. This provides a comprehensive view of how reconciliation affects forecast accuracy across different levels of the hierarchy, allowing for a detailed assessment of its effectiveness.

## MiddleOut

In [19]:
reconcilers = [
    MiddleOut(middle_level = 'top_level/middle_level', top_down_method='forecast_proportions')
]
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)

# column name of the reconciled forecasts
reconciled_col = Y_rec_df.columns[-1]

Y_rec_df.head(3)

,unique_id,ds,AutoARIMA,AutoARIMA/BottomUp
0,Total,2016-05-16,14.092568,13.943087
1,Total,2016-05-17,14.102201,13.327107
2,Total,2016-05-18,14.102201,13.409471


In [20]:
# Merge reconciled forecasts with test data
df_eval = merge_forecasts(Y_test_df, Y_rec_df)

In [21]:
metric = 'rmse'

baseline_overall = compute_overall_error(df_eval, actual_col, baseline_col, metric)
reconciled_overall = compute_overall_error(df_eval, actual_col, reconciled_col, metric)

print(f"Baseline {metric.upper()}: {baseline_overall:.4f}")
print(f"Reconciled {metric.upper()}: {reconciled_overall:.4f}")
print(f"Delta {metric.upper()}: {reconciled_overall - baseline_overall:.4f}")

Baseline RMSE: 2.1500
Reconciled RMSE: 2.1800
Delta RMSE: 0.0300


In [22]:
per_level_baseline = compute_error_per_level(df_eval, tags, actual_col, baseline_col, metric)
per_level_reconciled = compute_error_per_level(df_eval, tags, actual_col, reconciled_col, metric)


comparison_df = compare_baseline_vs_reconciled(
    df=df_eval,
    tags=tags,
    actual_col=actual_col,
    baseline_col=baseline_col,
    reconciled_col=reconciled_col,
    metric=metric
)

comparison_df

,level,baseline_rmse,reconciled_rmse,delta_rmse,improved
0,top_level,5.06,5.14,0.08,False
1,top_level/middle_level,2.45,2.52,0.07,False
2,top_level/middle_level/bottom_level,1.45,1.45,0.00,False


## TopDown

In [23]:
reconcilers = [
   TopDown(method='forecast_proportions')]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)

# column name of the reconciled forecasts
reconciled_col = Y_rec_df.columns[-1]

Y_rec_df.head(3)

,unique_id,ds,AutoARIMA,AutoARIMA/TopDown_method-forecast_proportions
0,Total,2016-05-16,14.092568,14.092568
1,Total,2016-05-17,14.102201,14.102201
2,Total,2016-05-18,14.102201,14.102201


In [24]:
# Merge reconciled forecasts with test data
df_eval = merge_forecasts(Y_test_df, Y_rec_df)

In [25]:
metric = 'rmse'

baseline_overall = compute_overall_error(df_eval, actual_col, baseline_col, metric)
reconciled_overall = compute_overall_error(df_eval, actual_col, reconciled_col, metric)

print(f"Baseline {metric.upper()}: {baseline_overall:.4f}")
print(f"Reconciled {metric.upper()}: {reconciled_overall:.4f}")
print(f"Delta {metric.upper()}: {reconciled_overall - baseline_overall:.4f}")

Baseline RMSE: 2.1500
Reconciled RMSE: 2.1400
Delta RMSE: -0.0100


In [26]:
per_level_baseline = compute_error_per_level(df_eval, tags, actual_col, baseline_col, metric)
per_level_reconciled = compute_error_per_level(df_eval, tags, actual_col, reconciled_col, metric)


comparison_df = compare_baseline_vs_reconciled(
    df=df_eval,
    tags=tags,
    actual_col=actual_col,
    baseline_col=baseline_col,
    reconciled_col=reconciled_col,
    metric=metric
)

comparison_df

,level,baseline_rmse,reconciled_rmse,delta_rmse,improved
0,top_level,5.06,5.06,0.00,False
1,top_level/middle_level,2.45,2.43,-0.02,True
2,top_level/middle_level/bottom_level,1.45,1.44,-0.01,True


## MinTrace

In [27]:
reconcilers = [
    MinTrace(method='ols'),
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)


Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)

# column name of the reconciled forecasts
reconciled_col = Y_rec_df.columns[-1]

Y_rec_df.head(3)



,unique_id,ds,AutoARIMA,AutoARIMA/MinTrace_method-ols
0,Total,2016-05-16,14.092568,14.050535
1,Total,2016-05-17,14.102201,13.992761
2,Total,2016-05-18,14.102201,13.970089


In [28]:
# Merge reconciled forecasts with test data
df_eval = merge_forecasts(Y_test_df, Y_rec_df)

In [29]:
metric = 'rmse'

baseline_overall = compute_overall_error(df_eval, actual_col, baseline_col, metric)
reconciled_overall = compute_overall_error(df_eval, actual_col, reconciled_col, metric)

print(f"Baseline {metric.upper()}: {baseline_overall:.4f}")
print(f"Reconciled {metric.upper()}: {reconciled_overall:.4f}")
print(f"Delta {metric.upper()}: {reconciled_overall - baseline_overall:.4f}")

Baseline RMSE: 2.1500
Reconciled RMSE: 2.1500
Delta RMSE: 0.0000


In [30]:
per_level_baseline = compute_error_per_level(df_eval, tags, actual_col, baseline_col, metric)
per_level_reconciled = compute_error_per_level(df_eval, tags, actual_col, reconciled_col, metric)


comparison_df = compare_baseline_vs_reconciled(
    df=df_eval,
    tags=tags,
    actual_col=actual_col,
    baseline_col=baseline_col,
    reconciled_col=reconciled_col,
    metric=metric
)

comparison_df

,level,baseline_rmse,reconciled_rmse,delta_rmse,improved
0,top_level,5.06,5.07,0.01,False
1,top_level/middle_level,2.45,2.45,0.00,False
2,top_level/middle_level/bottom_level,1.45,1.44,-0.01,True
